# Python program to implement webcam motion detector

This code is adapted from - https://www.geeksforgeeks.org/webcam-motion-detector-python/
Video demo can be found at - https://drive.google.com/file/d/1VHFYPTDxZunDT95b8dLM1dSQ3rAGxZWM/view?usp=sharing


In [ ]:
# importing OpenCV, time and Pandas library
import cv2, time, pandas
# importing datetime class from datetime library
from datetime import datetime

In [ ]:
# Assigning our static_back to None
static_back = None
 
# List when any moving object appear
motion_list = [ None, None ]
 
# Time of movement
time = []

In [ ]:
# Initializing DataFrame, one column is start
# time and other column is end time
df = pandas.DataFrame(columns = ["Start", "End"])
print(time)


### The command below determines which camera will be used as commented. 

In [ ]:
camera_to_use=1; #1 for external camera and 0 for laptop camera

### The value below should be less than or equal to 255. The more the number is after, the more regions in the thresold frame will be black. This need to be tweaked based on the lighting condition to get good motion detection.

In [1]:
detection_thresold=80;

In [ ]:
# Capturing video
video = cv2.VideoCapture(camera_to_use)
 
# Infinite while loop to treat stack of image as video
while True:
    # Reading frame(image) from video
    check, frame = video.read()

    # Initializing motion = 0(no motion)
    motion = 0
 
    # Converting color image to gray_scale image
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
 
    # Converting gray scale image to GaussianBlur
    # so that change can be find easily
    gray = cv2.GaussianBlur(gray, (21, 21), 0)
 
    # In first iteration we assign the value
    # of static_back to our first frame
    if static_back is None:
        static_back = gray
        continue
 
    # Difference between static background
    # and current frame(which is GaussianBlur)
    diff_frame = cv2.absdiff(static_back, gray)
 
    # If change in between static background and
    # current frame is greater than detection_thresold it will show white color(255)
    thresh_frame = cv2.threshold(diff_frame, detection_thresold, 255, cv2.THRESH_BINARY)[1]
    thresh_frame = cv2.dilate(thresh_frame, None, iterations = 2)
 
    # Finding contour of moving object
    cnts,_ = cv2.findContours(thresh_frame.copy(),
                       cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
 
    for contour in cnts:
        if cv2.contourArea(contour) < 10000:
            continue
        motion = 1
 
        (x, y, w, h) = cv2.boundingRect(contour)
        # making green rectangle around the moving object
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 3)
 
    # Appending status of motion
    motion_list.append(motion)
 
    motion_list = motion_list[-2:]
 
    # Appending Start time of motion
    if motion_list[-1] == 1 and motion_list[-2] == 0:
        time.append(datetime.now()) #Appending the current time
 
    # Appending End time of motion
    if motion_list[-1] == 0 and motion_list[-2] == 1:
        time.append(datetime.now())
 
    # Displaying image in gray_scale
    cv2.imshow("Gray Frame", gray)
 
    # Displaying the difference in currentframe to
    # the staticframe(very first_frame)
    cv2.imshow("Difference Frame", diff_frame)
 
    # Displaying the black and white image in which if
    # intensity difference greater than 30 it will appear white
    cv2.imshow("Threshold Frame", thresh_frame)
 
    # Displaying color frame with contour of motion of object
    cv2.imshow("Color Frame", frame)
 
    key = cv2.waitKey(1)
    # if q entered whole process will stop
    if key == ord('q'):
        # if something is movingthen it append the end time of movement
        if motion == 1:
            time.append(datetime.now())
        break

#Appending time of motion in DataFrame
#If no motion is captured the code would say so
if len(time)<2:
    print("No motion captured")
else:
    for i in range(0, len(time), 2):
        df = df.append({"Start":time[i], "End":time[i + 1]}, ignore_index = True)

# Creating a CSV file in which time of movements will be saved
df.to_csv("Time_of_movements.csv")
video.release()
 
# Destroying all the windows
cv2.destroyAllWindows()     
    

### We now define a new file to store time of motion so that the date and time can be seperated

In [ ]:
#Defining a new file to store time stamps
df2 = pandas.DataFrame(columns = ["Start Date","Start Time", "End Date","End Time"])
if len(time)<2:
    print("No motion captured")
else:
    for i in range(0, len(time), 2):
        tsd= str(time[i].year)+"/"+str(time[i].month)+"/"+str(time[i].day)#start date
        tst= str(time[i].hour)+":"+str(time[i].minute)+":"+str(time[i].second)#start time 
        
        ted= str(time[i+1].year)+"/"+str(time[i+1].month)+"/"+str(time[i+1].day)#end date
        tet= str(time[i+1].hour)+":"+str(time[i+1].minute)+":"+str(time[i+1].second)#end time
        df2 = df2.append({"Start Date":tsd,"Start Time":tst, "End Date":ted,"End Time":tet}, ignore_index = True)
# Creating a CSV file in which time of movements will be saved
df2.to_csv("Time_and_date_of_movements.csv")
video.release()
 
# Destroying all the windows
cv2.destroyAllWindows()    

In [ ]:
time

In [ ]:
tsd

In [ ]:
time[0].day

In [ ]:
len(time)